In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.6 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!pip install gdown

In [4]:
!gdown --id 1EZ8I39Xptum68PLF2LTDR7DO3X48TxN3

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1EZ8I39Xptum68PLF2LTDR7DO3X48TxN3
From (redirected): https://drive.google.com/uc?id=1EZ8I39Xptum68PLF2LTDR7DO3X48TxN3&confirm=t&uuid=6dc72404-6b65-4650-aecb-38625f1ab1b5
To: /kaggle/working/SARD_dataset.zip
100%|███████████████████████████████████████| 2.82G/2.82G [00:26<00:00, 105MB/s]


In [5]:
!pip install ultralytics -q
!pip install torch torchvision -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 20.1 MB/s eta 0:00:00


In [6]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=f6d1b8e18f7e95e6f93dd39903d1d89f33766db40c9d8a36e20da0fcf536d9f1
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [7]:
import zipfile
import os
import shutil
import timm
import random
from ultralytics import YOLO
import torch

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [9]:
# Define paths
zip_path = "/kaggle/working/SARD_dataset.zip"  # Update with your zip file path
extract_path = "/kaggle/working/dataset"  # Destination for extracted files

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [10]:
# Define paths
src_image_dir = "/kaggle/working/dataset/SARD_dataset/image"
src_label_dir = "/kaggle/working/dataset/SARD_dataset/label"
dest_dir = "/kaggle/working/data"

# Create new directory structure
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(dest_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(dest_dir, "labels", split), exist_ok=True)

# Get all images and shuffle them
images = [f for f in os.listdir(src_image_dir) if f.endswith(('.jpg', '.png'))]
random.shuffle(images)

# Split data (80% train, 10% val, 10% test)
train_split = int(0.8 * len(images))
val_split = int(0.9 * len(images))

train_images = images[:train_split]
val_images = images[train_split:val_split]
test_images = images[val_split:]

def move_files(file_list, split):
    for file in file_list:
        img_src = os.path.join(src_image_dir, file)
        lbl_src = os.path.join(src_label_dir, file.replace('.jpg', '.txt').replace('.png', '.txt'))

        img_dest = os.path.join(dest_dir, "images", split, file)
        lbl_dest = os.path.join(dest_dir, "labels", split, os.path.basename(lbl_src))

        shutil.move(img_src, img_dest)
        if os.path.exists(lbl_src):
            shutil.move(lbl_src, lbl_dest)

# Move files
move_files(train_images, "train")
move_files(val_images, "val")
move_files(test_images, "test")

print("Dataset reorganization complete.")


Dataset reorganization complete.


In [11]:
# Define dataset path
DATASET_PATH = "/kaggle/working/data"

# YOLO dataset configuration
dataset_config = {
    'path': DATASET_PATH,
    'train': os.path.join('images', 'train'),
    'val': os.path.join('images', 'val'),
    'names': {
        0: 'person'  # Assuming 'person' is the only class
    }
}

In [12]:
# Save dataset configuration as a YAML file
import yaml

config_path = os.path.join(DATASET_PATH, "data.yaml")
with open(config_path, 'w') as file:
    yaml.dump(dataset_config, file, default_flow_style=False)

print(f"Dataset configuration saved to {config_path}")

Dataset configuration saved to /kaggle/working/data/data.yaml


In [ ]:
# Load YOLOv10 model
model = YOLO("yolov10m.pt")  # YOLOv10m (Medium) model used
# model = YOLO("yolov10n.pt")  # YOLOv10n (Nano) model
# model = YOLO("yolov10s.pt")  # YOLOv10s (Small) model
# model = YOLO("yolov10l.pt")  # YOLOv10l (Large) model
# model = YOLO("yolov10x.pt")  # YOLOv10x (Extra Large) model

# Load EfficientNetV2-M model without pretrained weights
efficientnet_v2_m = timm.create_model("efficientnetv2_m", pretrained=False).to(device)

# Replace YOLO backbone with EfficientNetV2-M
model.model.model[0] = efficientnet_v2_m  # EfficientNetV2-M used
# model.model.model[0] = timm.create_model("efficientnetv2_l", pretrained=False).to(device)  # EfficientNetV2-L
# model.model.model[0] = timm.create_model("efficientnetv2_s", pretrained=False).to(device)  # EfficientNetV2-S
# model.model.model[0] = timm.create_model("efficientnet_b7", pretrained=False).to(device)  # EfficientNet-B7
# model.model.model[0] = models.mobilenet_v2(pretrained=True).features  # MobileNetV2
# model.model.model[0] = models.squeezenet1_1(pretrained=True).features  # SqueezeNet
# model.model.model[0] = models.densenet121(pretrained=True).features  # DenseNet-121


100%|██████████| 32.1M/32.1M [00:00<00:00, 109MB/s] 


In [ ]:
model.train(data=config_path, epochs=100, batch=16, imgsz=640)

Ultralytics 8.3.84 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov10m.pt, data=/kaggle/working/data/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 26.3MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1     78720  ultralytics.nn.modules.block.SCDown          [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1    228672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 97.9MB/s]


AMP: checks passed ✅


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     45/100      9.74G      2.922        1.3      1.765         82        640: 100%|██████████| 99/99 [01:09<00:00,  1.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  3.22it/s]

                   all        198        689       0.91      0.821      0.894      0.535



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     46/100       9.7G      2.853      1.231      1.763        102        640:  82%|████████▏ | 81/99 [00:57<00:12,  1.42it/s]

In [ ]:
# Validate trained model
metrics = model.val()
print(metrics)


In [ ]:
from ultralytics import YOLO

# Load the trained model
model_path = "/kaggle/working/runs/detect/train/weights/best.pt"  # Update with correct path
model = YOLO(model_path)

print("Model Loaded Successfully!")

In [ ]:
!zip -r best_model.zip /kaggle/working/runs/detect/train/weights/best.pt

In [ ]:
%cd /kaggle/working
!ls
from IPython.display import FileLink
FileLink(r'best_model.zip')